## **Определение тональности отзывов на банки с помощью векторных представлений слов**
### *Курс Андрея Созыкина, "Обработка естественного языка", видео "Плотные векторные представления слов для определение тональности"*

In [1]:
!pip install pymorphy2 -qq
!pip install navec -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 53.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from navec import Navec

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
MAX_REVIEW_LEN = 100
VECTOR_SIZE = 300

TEST_SIZE = 0.2
RANDOM_STATE = 42
MAX_ITER = 500

### Загрузка и подготовка набора данных

In [9]:
banks = pd.read_csv('banks.csv', sep='\t', index_col='idx')

In [10]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [11]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [12]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [13]:
banks.head()

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан..."


### Загрузка предварительно обученных векторов Navec

In [14]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar -qq

In [15]:
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

### Векторизация текста

In [16]:
def vectorize_text(txt, navec, max_review_len):
    unk = navec['<unk>']
    text_embeddings = []
    for tocken in txt:
        embedding = navec.get(tocken, unk)
        text_embeddings.append(embedding)
    # Дополняем или обрезаем отзывы для фиксированной длины max_review_len 
    l = len(text_embeddings)
    if l > max_review_len:
        text_embeddings = text_embeddings[:max_review_len]
    else:
        text_embeddings.extend([navec['<pad>']] * (max_review_len - l)) 
    return text_embeddings

In [17]:
banks['Embeddings'] = banks.apply(lambda row: vectorize_text(row['Preprocessed_texts'], navec, MAX_REVIEW_LEN), axis=1)

In [18]:
banks.head()

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[[-0.038867943, 0.12470589, 0.45623106, -0.037..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[[0.0018760362, 0.47031733, 0.44177133, 0.2619..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[[-0.28339294, 0.14809653, -0.08746451, 0.3650..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[[-0.2938254, -0.2968786, -0.025342526, 0.0822..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[[0.36390617, -0.5678413, -0.24011786, 0.08220..."


### Подготовка данных для обучения

In [19]:
mapping = {'Negative': 0, 'Positive': 1}
banks.replace({'Score': mapping}, inplace=True)

In [20]:
banks.head()

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[[-0.038867943, 0.12470589, 0.45623106, -0.037..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[[0.0018760362, 0.47031733, 0.44177133, 0.2619..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[[-0.28339294, 0.14809653, -0.08746451, 0.3650..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[[-0.2938254, -0.2968786, -0.025342526, 0.0822..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[[0.36390617, -0.5678413, -0.24011786, 0.08220..."


### Выделение данных для обучения и тестирования

In [22]:
train, test = train_test_split(banks, test_size=TEST_SIZE)

In [23]:
train.head(3)

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
7980,0,19.11.2009 г. оплачивал билеты на поезд с Пите...,"[19.11.2009, г., оплачивать, билет, поезд, пит...","[[0.21431214, 0.37028718, 0.13679631, -0.18653..."
3163,0,Между мной и Альфа -банком был заключен кредит...,"[я, альфа, -банк, заключить, кредитный, догово...","[[-0.34992936, -0.3075621, -0.27499628, 0.0011..."
12758,1,Порядка двух лет был вкладчиком банка. Так как...,"[порядок, два, год, вкладчик, банк, сумма, кру...","[[0.057401087, -0.69955, 0.550758, -0.00319764..."


In [24]:
test.head(3)

,Score,Text,Preprocessed_texts,Embeddings
idx,,,,
7512,1,Для размещения денежных средств моя семья поль...,"[размещение, денежный, средство, семья, пользо...","[[0.49163, 0.003914773, -0.034092356, 0.122450..."
9816,0,Каждую неделю с Альфа банка приходят рассылки ...,"[каждый, неделя, альфа, банк, приходить, рассы...","[[0.053723477, -0.0813841, -0.51375115, 0.4728..."
2683,1,Открывали расчетный счет в отделении в г. Мыти...,"[открывать, расчётный, счёт, отделение, г., мы...","[[-0.1191054, -0.16411726, -0.36560267, 0.5830..."


### Разделение меток классов и данных для обучения

In [25]:
x_train = np.array(train['Embeddings'].tolist()).reshape(len(train), VECTOR_SIZE * MAX_REVIEW_LEN)
y_train = train['Score']

In [27]:
x_test = np.array(test['Embeddings'].tolist()).reshape(len(test), VECTOR_SIZE * MAX_REVIEW_LEN)
y_test = test['Score']

In [28]:
x_train.shape

(11199, 30000)

In [29]:
x_test.shape

(2800, 30000)

### Создание и обучение модели машинного обучения

In [32]:
lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=MAX_ITER)

In [33]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

In [34]:
lr.score(x_test, y_test)

0.7910714285714285

### Применение модели для определения тональности отзыва на банк. Позитивный отзыв

In [47]:
positive_text = """Брал кредит в ОченьХорошемБанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат. 
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж. 
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""

In [55]:
positive_preprocessed_text = preprocess(positive_text, stop_words, punctuation_marks, morph)
positive_vectorized_text = vectorize_text(positive_preprocessed_text, navec, MAX_REVIEW_LEN)
positive_vector = np.array(positive_vectorized_text).reshape(1, VECTOR_SIZE * MAX_REVIEW_LEN)
result = lr.predict(positive_vector)
result

array([1])

### Применение модели для определения тональности отзыва на банк. Негативный отзыв

In [44]:
negative_text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям 
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""

In [45]:
negative_preprocessed_text = preprocess(negative_text, stop_words, punctuation_marks, morph)
negative_vectorized_text = vectorize_text(negative_preprocessed_text, navec, MAX_REVIEW_LEN)
negative_vector = np.array(negative_vectorized_text).reshape(1, VECTOR_SIZE * MAX_REVIEW_LEN)
result = lr.predict(negative_vector)
result

array([0])

In [46]:
result = lr.predict_proba(negative_vector)
result

array([[0.97636811, 0.02363189]])